In [4]:
# imports
import snntorch as snn
from snntorch import surrogate
from snntorch import backprop
from snntorch import functional as SF
from snntorch import utils
from snntorch import spikeplot as splt
from snntorch import spikegen
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.nn.parameter import Parameter
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np
import itertools

import os
import shutil

# dataloader arguments
batch_size = 128
# data_path='~/justinData/mnist'
subset=10

dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

spike_grad = surrogate.fast_sigmoid(slope=25)
beta = 0.5

lif1 = snn.Leaky(beta=beta, spike_grad=spike_grad)

#  Initialize Network
net = nn.Sequential(nn.Conv2d(1, 12, 5),
                    nn.AvgPool2d(2, stride=2),
                    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True),
                    nn.Conv2d(12, 64, 5),
                    nn.AvgPool2d(2, stride=2),
                    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True),
                    nn.Flatten(),
                    nn.Linear(64*4*4, 10),
                    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True)
                    ).to(device)

In [5]:
best_model_path = 'model_cifar_39_s.pth.tar'
checkpoint = torch.load(best_model_path, map_location=torch.device('cpu'))
print(checkpoint)

{'state_dict': OrderedDict([('conv_fc.0.weight', tensor([[[[ 0.0931,  0.0193, -0.0849],
          [ 0.1109,  0.2121, -0.2605],
          [ 0.0376, -0.0058, -0.0403]],

         [[-0.0475,  0.1391, -0.1495],
          [ 0.2059,  0.1943, -0.3784],
          [ 0.0901,  0.0615, -0.2394]],

         [[ 0.1880, -0.1464, -0.0732],
          [ 0.0828,  0.1378, -0.3004],
          [ 0.2835, -0.0221, -0.1532]]],


        [[[-0.0157, -0.1529,  0.1716],
          [ 0.0594, -0.1340, -0.0643],
          [ 0.0184, -0.1188,  0.0965]],

         [[ 0.1957, -0.0544, -0.0770],
          [-0.0348, -0.0664,  0.0313],
          [-0.0026, -0.0582,  0.1344]],

         [[-0.0316,  0.0745, -0.0949],
          [-0.1542, -0.1321,  0.1678],
          [-0.2048, -0.0533,  0.2732]]],


        [[[ 0.1676, -0.2314, -0.2622],
          [ 0.1440, -0.2259, -0.1999],
          [ 0.2448,  0.0399,  0.2511]],

         [[ 0.1769, -0.1979,  0.1028],
          [-0.1555, -0.0954,  0.0448],
          [ 0.1123,  0.0420,  0.0193

In [51]:
layers = [net.state_dict()['0.weight'].detach().cpu().numpy(),net.state_dict()['3.weight'].detach().cpu().numpy(),net.state_dict()['7.weight'].detach().cpu().numpy()]
biases = [net.state_dict()['0.bias'].detach().cpu().numpy(),net.state_dict()['3.bias'].detach().cpu().numpy(),net.state_dict()['7.bias'].detach().cpu().numpy()]

import numpy as np
print(np.min(layers[1]))
print(np.max(layers[1]))

def conv2dOutputSize(layer,inputSize):
    H_out = (inputSize[0] + layer.padding[0]-layer.dilation[0]*(layer.kernel_size[0]-1)-1)/layer.stride[0] +1
    W_out = (inputSize[1] + layer.padding[0]-layer.dilation[1]*(layer.kernel_size[1]-1)-1)/layer.stride[1] +1
    return [layer.out_channels,int(H_out),int(W_out)]

def maxPoolOutputSize(layer,inputSize):
    H_out = (inputSize[1] + layer.padding - layer.dilation*(layer.kernel_size-1)-1)/layer.stride +1
    W_out = (inputSize[2] + layer.padding - layer.dilation*(layer.kernel_size-1)-1)/layer.stride +1
    return [inputSize[0],int(H_out),int(W_out)]

def AvgPoolOutputSize(layer,inputSize):
    H_out = (inputSize[1] + layer.padding*2 - (layer.kernel_size-1))/layer.stride +1
    W_out = (inputSize[2] + layer.padding*2 - (layer.kernel_size-1))/layer.stride +1
    return [inputSize[0],int(H_out),int(W_out)]


def optimized_conv2dToCRI(inputs, output, layer, layerIdx, axonsDict=None, neuronsDict=None):
    Hk, Wk = layer.kernel_size
    filters = layer.weight.detach().cpu().numpy()
    pad_top, pad_left = Hk // 2, Wk // 2

    # Define a helper function to reduce code redundancy
    def process_patch(input_slice, output_slice, filters, axonsDict, neuronsDict, layerIdx, channel=None):
        """
        This function processes a given patch of input neurons or axons and 
        updates the axonsDict or neuronsDict accordingly.
        """
        for filIdx, fil in enumerate(filters):
            postSynapticID = str(output_slice[filIdx])
            fil = fil[channel] if channel is not None else fil[0]
            for i, row in enumerate(input_slice):
                for j, elem in enumerate(row):
                    key = str(elem) if layerIdx != 0 else elem
                    synapse_info = (postSynapticID, int(fil[i, j]))
                    (neuronsDict if layerIdx != 0 else axonsDict)[key].append(synapse_info)

    if layerIdx == 0:
        Hi, Wi = inputs.shape
        for row in range(pad_top, Hi - pad_top):
            for col in range(pad_left, Wi - pad_left):
                patch = inputs[row - pad_top:row + pad_top + 1, col - pad_left:col + pad_left + 1]
                output_slice = output[:, row - pad_top, col - pad_left]
                process_patch(patch, output_slice, filters, axonsDict, neuronsDict, layerIdx)
    else:
        C, Hi, Wi = inputs.shape
        for channel in range(C):
            for row in range(pad_top, Hi - pad_top):
                for col in range(pad_left, Wi - pad_left):
                    patch = inputs[channel, row - pad_top:row + pad_top + 1, col - pad_left:col + pad_left + 1]
                    output_slice = output[:, row - pad_top, col - pad_left]
                    process_patch(patch, output_slice, filters, axonsDict, neuronsDict, layerIdx, channel)

def poolingToCRI(inputs, output, layer, neuronsDict, scaler=1e6, pool_type='max'):
    C, Hi, Wi = inputs.shape
    kernel_size = layer.kernel_size
    pad_size = kernel_size // 2
    stride = 2

    for channel in range(C):
        for row in range(0, Hi, stride):
            for col in range(0, Wi, stride):
                patch = inputs[channel, row:row+kernel_size, col:col+kernel_size]
                postSynapticID = str(output[channel, row//stride, col//stride])

                # Determine the pooling value
                if pool_type == 'max':
                    pooling_value = patch.max() * scaler
                elif pool_type == 'avg':
                    pooling_value = patch.mean() * scaler

                # Update neuronsDict with postSynapticID and pooling_value
                for preSynNeuron in patch.flatten():
                    neuronsDict[str(preSynNeuron)].append((postSynapticID, pooling_value))

# Wrapper functions for max pooling and average pooling
def maxPoolToCRI(inputs, output, layer, neuronsDict, scaler=1e6):
    return poolingToCRI(inputs, output, layer, neuronsDict, scaler, pool_type='max')

def avgPoolToCRI(inputs, output, layer, neuronsDict, scaler=1e6):
    return poolingToCRI(inputs, output, layer, neuronsDict, scaler, pool_type='avg')

  
def linearToCRI(inputs,output,layer,layerIdx,neuronsDict,outputNeurons=None):
    inputs = inputs.flatten()
    weight = layer.weight.detach().cpu().numpy()
    currLayerNeuronIdxOffset,nextLayerNeuronIdxOffset = inputs[0],inputs[-1]+1
    for baseNeuronIdx, neuron in enumerate(weight.T):
        neuronID = str(baseNeuronIdx+currLayerNeuronIdxOffset)
        neuronEntry = [(str(basePostSynapticID+nextLayerNeuronIdxOffset), int(synapseWeight)) for basePostSynapticID, synapseWeight in enumerate(neuron) if synapseWeight != 0]
        neuronsDict[neuronID] = neuronEntry
    print('instantiate output neurons')
    for baseNeuronIdx in range(layer.out_features):
        neuronID = str(baseNeuronIdx+nextLayerNeuronIdxOffset)
        neuronsDict[neuronID] = []
        outputNeurons.append(neuronID)
        
def convBiasAxons(layer,axonsDict,axonOffset,outputs):
    biases = layer.bias.detach().cpu().numpy()
    for biasIdx, bias in enumerate(biases):
        biasID = 'a'+str(biasIdx+axonOffset)
        axonsDict[biasID] = [(str(neuronIdx),int(bias)) for neuronIdx in outputs[biasIdx].flatten()]
        
def linearBiasAXons(layer,axonsDict,axonOffset,outputs):
    biases = layer.bias.detach().cpu().numpy()
    for biasIdx, bias in enumerate(biases):
        biasID = 'a'+str(biasIdx+axonOffset)
        axonsDict[biasID] = [(str(outputs[biasIdx]),int(bias))]
        

-0.057706498
0.05773118


In [25]:
class CRIConverter:
    def __init__(self, scalar=1e6):
        self.axonsDict = {}
        self.neuronsDict = {}
        self.scalar = scalar
        
    @staticmethod 
    def _conv2dOutputSize(layer, inputSize):
        H_out = (inputSize[0] + 2 * layer.padding[0] - layer.dilation[0] * (layer.kernel_size[0] - 1) - 1) // layer.stride[0] + 1
        W_out = (inputSize[1] + 2 * layer.padding[1] - layer.dilation[1] * (layer.kernel_size[1] - 1) - 1) // layer.stride[1] + 1
        return [layer.out_channels, int(H_out), int(W_out)]
    
    @staticmethod
    def _maxPoolOutputSize(layer, inputSize):
        H_out = (inputSize[1] + 2 * layer.padding - layer.dilation * (layer.kernel_size - 1) - 1) // layer.stride + 1
        W_out = (inputSize[2] + 2 * layer.padding - layer.dilation * (layer.kernel_size - 1) - 1) // layer.stride + 1
        return [inputSize[0], int(H_out), int(W_out)]
    
    @staticmethod
    def _avgPoolOutputSize(layer, inputSize):
        H_out = (inputSize[1] + layer.padding*2 - (layer.kernel_size-1))/layer.stride +1
        W_out = (inputSize[2] + layer.padding*2 - (layer.kernel_size-1))/layer.stride +1
        return [inputSize[0],int(H_out),int(W_out)]   
    
    def conv2dToCRI(self, inputs, output, layer, layerIdx, axonsDict=None, neuronsDict=None):
        """
        Convert a convolutional layer to a CRI representation.
        
        Parameters:
            inputs (torch.Tensor): The input tensor to the convolutional layer.
            output (torch.Tensor): The output tensor of the convolutional layer.
            layer (torch.nn.Conv2d): The convolutional layer to convert.
            layerIdx (int): The index of the current convolutional layer within the model.
            axonsDict (dict, optional): A dictionary that maps axon IDs to their synapse information. Defaults to None.
            neuronsDict (dict, optional): A dictionary that maps neuron IDs to their synapse information. Defaults to None.
        
        Returns:
            None
        """
            
        Hk, Wk = layer.kernel_size
        filters = layer.weight.detach().cpu().numpy()
        pad_top, pad_left = Hk // 2, Wk // 2

        # Define a helper function to reduce code redundancy
        def process_patch(input_slice, output_slice, filters, axonsDict, neuronsDict, layerIdx, channel=None):
            """
            Process a given patch of input neurons or axons and update axonsDict or neuronsDict.
            """
            for filIdx, fil in enumerate(filters):
                postSynapticID = str(output_slice[filIdx])
                fil = fil[channel] if channel is not None else fil[0]
                for i, row in enumerate(input_slice):
                    for j, elem in enumerate(row):
                        key = str(elem) if layerIdx != 0 else elem
                        synapse_info = (postSynapticID, int(fil[i, j]))
                        (neuronsDict if layerIdx != 0 else axonsDict)[key].append(synapse_info)
        
        if layerIdx == 0:
            Hi, Wi = inputs.shape
            for row in range(pad_top, Hi - pad_top):
                for col in range(pad_left, Wi - pad_left):
                    patch = inputs[row - pad_top:row + pad_top + 1, col - pad_left:col + pad_left + 1]
                    output_slice = output[:, row - pad_top, col - pad_left]
                    process_patch(patch, output_slice, filters, axonsDict, neuronsDict, layerIdx)
        else:
            C, Hi, Wi = inputs.shape
            for channel in range(C):
                for row in range(pad_top, Hi - pad_top):
                    for col in range(pad_left, Wi - pad_left):
                        patch = inputs[channel, row - pad_top:row + pad_top + 1, col - pad_left:col + pad_left + 1]
                        output_slice = output[:, row - pad_top, col - pad_left]
                        process_patch(patch, output_slice, filters, axonsDict, neuronsDict, layerIdx, channel)


    def poolingToCRI(self, inputs, output, neuronsDict, layer, pool_type='max'):
        """
        Convert a pooling layer to a CRI representation.
        
        Parameters:
            inputs (torch.Tensor): The input tensor to the pooling layer.
            output (torch.Tensor): The output tensor of the pooling layer.
            neuronsDict (dict): A dictionary that maps neuron IDs to their synapse information.
            layer (torch.nn.MaxPool2d or torch.nn.AvgPool2d): The pooling layer to convert.
            pool_type (str, optional): The type of pooling operation, either 'max' or 'avg'. Defaults to 'max'.

        Returns:
            None
        """
        C, Hi, Wi = inputs.shape
        kernel_size = layer.kernel_size
        pad_size = kernel_size // 2
        stride = 2

        for channel in range(C):
            for row in range(0, Hi, stride):
                for col in range(0, Wi, stride):
                    patch = inputs[channel, row:row+kernel_size, col:col+kernel_size]
                    postSynapticID = str(output[channel, row//stride, col//stride])

                    # Determine the pooling value
                    if pool_type == 'max':
                        pooling_value = patch.max() * self.scalar
                    elif pool_type == 'avg':
                        pooling_value = patch.mean() * self.scalar

                    # Update neuronsDict with postSynapticID and pooling_value
                    for preSynNeuron in patch.flatten():
                        neuronsDict[str(preSynNeuron)].append((postSynapticID, pooling_value))
    
    # Wrapper functions
    def maxPoolToCRI(self, inputs, output, neuronsDict, layer):
        return self.poolingToCRI(inputs, output, neuronsDict, layer, pool_type='max')

    def avgPoolToCRI(self, inputs, output,neuronsDict, layer):
        return self.poolingToCRI(inputs, output, neuronsDict, layer, pool_type='avg')
    
    def linearToCRI(self, inputs, layer, neuronsDict, outputNeurons=None):
        """
        Convert a linear layer to a CRI representation.
        
        Parameters:
            inputs (torch.Tensor): The input tensor to the linear layer.
            output (torch.Tensor): The output tensor of the linear layer.
            layer (torch.nn.Linear): The linear layer to convert.
            layerIdx (int): The index of the current linear layer within the model.
            neuronsDict (dict): A dictionary that maps neuron IDs to their synapse information.
            outputNeurons (list, optional): A list to store the neuron IDs of the output neurons.
                                        Defaults to None, in which case a new list is created.

        Returns:
            None
        """
        if outputNeurons is None:
            outputNeurons = []

        # Flatten the input tensor
        inputs = inputs.flatten()

        # Detach the weight matrix from the computational graph and convert to NumPy array
        weight = layer.weight.detach().cpu().numpy()

        # Determine neuron index offsets for the current and next layers
        currLayerNeuronIdxOffset, nextLayerNeuronIdxOffset = inputs[0], inputs[-1] + 1

        # Iterate over neurons and weights in the transposed weight matrix
        for baseNeuronIdx, neuron in enumerate(weight.T):
            neuronID = str(baseNeuronIdx + currLayerNeuronIdxOffset)
            # Create a list of synapses with non-zero weights
            neuronEntry = [(str(basePostSynapticID + nextLayerNeuronIdxOffset), int(synapseWeight))
                        for basePostSynapticID, synapseWeight in enumerate(neuron) if synapseWeight != 0]
            neuronsDict[neuronID] = neuronEntry

        # Instantiate output neurons
        for baseNeuronIdx in range(layer.out_features):
            neuronID = str(baseNeuronIdx + nextLayerNeuronIdxOffset)
            neuronsDict[neuronID] = []
            outputNeurons.append(neuronID)
    
    def convBiasAxons(self, layer, axonsDict, axonOffset, outputs):
        """
        Convert biases of a convolutional layer to axon entries in the CRI representation.
        
        Parameters:
            layer (torch.nn.Conv2d): The convolutional layer to convert.
            axonsDict (dict): A dictionary that maps axon IDs to their synapse information.
            axonOffset (int): An offset value for axon IDs.
            outputs (torch.Tensor): The output tensor of the convolutional layer.

        Returns:
            None
        """
        # Detach the biases from the computational graph and convert to NumPy array
        biases = layer.bias.detach().cpu().numpy()

        # Iterate over biases and create axon entries
        for biasIdx, bias in enumerate(biases):
            biasID = f'a{biasIdx + axonOffset}'
            axonsDict[biasID] = [(str(neuronIdx), int(bias)) for neuronIdx in outputs[biasIdx].flatten()]

    def linearBiasAxons(self, layer, axonsDict, axonOffset, outputs):
        """
        Convert biases of a linear layer to axon entries in the CRI representation.
        
        Parameters:
            layer (torch.nn.Linear): The linear layer to convert.
            axonsDict (dict): A dictionary that maps axon IDs to their synapse information.
            axonOffset (int): An offset value for axon IDs.
            outputs (torch.Tensor): The output tensor of the linear layer.

        Returns:
            None
        """
        # Detach the biases from the computational graph and convert to NumPy array
        biases = layer.bias.detach().cpu().numpy()

        # Iterate over biases and create axon entries
        for biasIdx, bias in enumerate(biases):
            biasID = f'a{biasIdx + axonOffset}'
            axonsDict[biasID] = [(str(outputs[biasIdx]), int(bias))]

In [41]:
from collections import defaultdict
import time
# import cProfile

# profiler = cProfile.Profile()

# Start profiling
# profiler.enable()
axonsDict = defaultdict(list)
neuronsDict = defaultdict(list)

# axonsDict1 = defaultdict(list)
# neuronsDict1 = defaultdict(list)
# axonsDict2 = defaultdict(list)
# neuronsDict2 = defaultdict(list)

outputNeurons = []
H_in, W_in = 28, 28
inputSize = (H_in, W_in)
axonOffset = 0
neuronOffset = 0
currInput = None
converter = CRIConverter()
    
    
for layerIdx, layer in enumerate(net):
    start_time = time.time()
        
    if layerIdx == 0 and isinstance(layer, torch.nn.Conv2d): #input layer
        print('constructing Axons')
        outputSize = converter._conv2dOutputSize(layer,inputSize)
        print("Input layer shape(infeature, outfeature): ", inputSize,',',outputSize)
        input = np.arange(0,inputSize[0]*inputSize[1],dtype=int).reshape(inputSize)
        inputAxons = np.array([['a'+str(i) for i in row] for row in input])
        output = np.arange(0,outputSize[0]*outputSize[1]*outputSize[2],dtype=int).reshape(outputSize)
        converter.conv2dToCRI(inputAxons,output,layer,layerIdx,axonsDict)
        axonOffset += len(axonsDict)
        print('constructing bias axons for input layer:',layer.bias.shape[0],'axons')
        converter.convBiasAxons(layer,axonsDict,axonOffset,output)
        axonOffset += layer.bias.shape[0]
        currInput = output
    elif layerIdx == len(net)-2 and isinstance(layer, torch.nn.Linear): #output layer
        print('constructing output layer')
        outputSize = layer.out_features
        print("output layer shape(infeature, outfeature): ", currInput.flatten().shape[0],',',outputSize)
        neuronOffset += currInput.shape[0]*currInput.shape[1]*currInput.shape[2]
        output = np.arange(neuronOffset,neuronOffset+outputSize,dtype=int)
        converter.linearToCRI(currInput,layer,neuronsDict,outputNeurons=outputNeurons)
        print('constructing bias axons for output linearlayer:',layer.bias.shape[0],'axons')
        print('Numer of neurons:',len(neuronsDict))
        converter.linearBiasAxons(layer,axonsDict,axonOffset,output)
        axonOffset += layer.bias.shape[0]
    else: #hidden layer
        if isinstance(layer,torch.nn.AvgPool2d):
            print('constructing hidden avgpool layer')
            outputSize = converter._avgPoolOutputSize(layer,currInput.shape)
            print("Hidden layer shape(infeature, outfeature): ", currInput.shape,',',outputSize)
            neuronOffset += currInput.shape[0]*currInput.shape[1]*currInput.shape[2]
            output = np.arange(neuronOffset,neuronOffset+outputSize[0]*outputSize[1]*outputSize[2],dtype=int).reshape(outputSize)
            converter.avgPoolToCRI(currInput,output,neuronsDict,layer)
            currInput = output
            print('Numer of neurons:',len(neuronsDict))
        if isinstance(layer,torch.nn.Conv2d):
            print('constructing hidden conv2d layer')
            outputSize = converter._conv2dOutputSize(layer,currInput.shape)
            print("Hidden layer shape(infeature, outfeature): ", currInput.shape,',',outputSize)
            neuronOffset += currInput.shape[0]*currInput.shape[1]*currInput.shape[2]
            output = np.arange(neuronOffset,neuronOffset+outputSize[0]*outputSize[1]*outputSize[2],dtype=int).reshape(outputSize)
            converter.conv2dToCRI(currInput,output,layer,layerIdx,neuronsDict=neuronsDict)
            print('constructing bias axons for hidden conv2d layer:',layer.bias.shape[0],'axons')
            converter.convBiasAxons(layer,axonsDict,axonOffset,output)
            axonOffset += layer.bias.shape[0]
            currInput = output
            print('Numer of neurons:',len(neuronsDict))
    end_time = time.time()
    
total_time = end_time-start_time
print(f'time taken: {total_time}s')

constructing Axons
Input layer shape(infeature, outfeature):  (28, 28) , [12, 24, 24]
constructing bias axons for input layer: 12 axons
constructing hidden avgpool layer
Hidden layer shape(infeature, outfeature):  (12, 24, 24) , [12, 12, 12]
Numer of neurons: 6912
constructing hidden conv2d layer
Hidden layer shape(infeature, outfeature):  (12, 12, 12) , [64, 8, 8]
constructing bias axons for hidden conv2d layer: 64 axons
Numer of neurons: 8640
constructing hidden avgpool layer
Hidden layer shape(infeature, outfeature):  (64, 8, 8) , [64, 4, 4]
Numer of neurons: 12736
constructing output layer
output layer shape(infeature, outfeature):  1024 , 10
constructing bias axons for output linearlayer: 10 axons
Numer of neurons: 13770
time taken: 2.1457672119140625e-06s


In [39]:
import torch
import torch.nn as nn
import unittest

class TestLayerToCRIConverter(unittest.TestCase):
    def setUp(self):
        # Create an instance of the LayerToCRIConverter class for testing
        self.converter = CRIConverter()

    def test_conv2dToCRI(self):
        # Test conv2dToCRI method with a simple convolutional layer
        conv_layer = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3, stride=1, padding=1)
        input_tensor = torch.randn(1, 1, 28, 28)  # Batch size: 1, Channels: 1, Height: 5, Width: 5
        output_tensor = conv_layer(input_tensor)
        neuronsDict = defaultdict(list)
        self.converter.conv2dToCRI(input_tensor, output_tensor, conv_layer, 0, neuronsDict=neuronsDict)

        # Assert that neuronsDict contains expected keys and synapse information
        self.assertIn('0', neuronsDict)  # Check if neuron ID '0' exists in neuronsDict
        self.assertIn('1', neuronsDict)  # Check if neuron ID '1' exists in neuronsDict

    # def test_maxPoolToCRI(self):
    #     # Test maxPoolToCRI method with a simple max pooling layer
    #     maxpool_layer = nn.MaxPool2d(kernel_size=2, stride=2)
    #     input_tensor = torch.tensor([[[[1.0, 2.0], [3.0, 4.0]]]])  # Batch size: 1, Channels: 1, Height: 2, Width: 2
    #     output_tensor = maxpool_layer(input_tensor)
    #     neuronsDict = {}
    #     self.converter.maxPoolToCRI(input_tensor, output_tensor, maxpool_layer, neuronsDict)

    #     # Assert that neuronsDict contains expected keys and pooling values
    #     self.assertIn('2', neuronsDict)  # Check if neuron ID '2' exists in neuronsDict
    #     self.assertEqual(neuronsDict['2'][0][1], 4.0)  # Check the pooling value

    # def test_avgPoolToCRI(self):
    #     # Test avgPoolToCRI method with a simple average pooling layer
    #     avgpool_layer = nn.AvgPool2d(kernel_size=2, stride=2)
    #     input_tensor = torch.tensor([[[[1.0, 2.0], [3.0, 4.0]]]])  # Batch size: 1, Channels: 1, Height: 2, Width: 2
    #     output_tensor = avgpool_layer(input_tensor)
    #     neuronsDict = defaultdict(list)
    #     self.converter.avgPoolToCRI(input_tensor, output_tensor, avgpool_layer, neuronsDict)

    #     # Assert that neuronsDict contains expected keys and pooling values
    #     self.assertIn('2', neuronsDict)  # Check if neuron ID '2' exists in neuronsDict
    #     self.assertEqual(neuronsDict['2'][0][1], 2.5)  # Check the pooling value

    # def test_linearToCRI(self):
    #     # Test linearToCRI method with a simple linear layer
    #     linear_layer = nn.Linear(in_features=4, out_features=2)
    #     input_tensor = torch.randn(1, 4) # Batch size: 1, Input features: 4
    #     output_tensor = linear_layer(input_tensor)
    #     neuronsDict = defaultdict(list)
    #     outputNeurons = []
    #     self.converter.linearToCRI(input_tensor, output_tensor, linear_layer, 0, neuronsDict, outputNeurons)
        
    #     # Assert that neuronsDict contains expected keys and synapse information
    #     self.assertIn('0', neuronsDict)  # Check if neuron ID '0' exists in neuronsDict
    #     self.assertIn('1', neuronsDict)  # Check if neuron ID '1' exists in neuronsDict
    #     self.assertIn('2', neuronsDict)  # Check if neuron ID '2' exists in neuronsDict
    #     self.assertIn('3', neuronsDict)  # Check if neuron ID '3' exists in neuronsDict

    #     # Assert that outputNeurons contains expected neuron IDs
    #     self.assertIn('4', outputNeurons)  # Check if neuron ID '4' exists in outputNeurons
    #     self.assertIn('5', outputNeurons)  # Check if neuron ID '5' exists in outputNeurons

    # def test_convBiasAxons(self):
    #     # Test convBiasAxons method with a simple convolutional layer
    #     conv_layer = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3, stride=1, padding=1)
    #     input_tensor = torch.randn(1, 1, 5, 5)  # Batch size: 1, Channels: 1, Height: 5, Width: 5
    #     output_tensor = conv_layer(input_tensor)
    #     axonsDict = {}
    #     self.converter.convBiasAxons(conv_layer, axonsDict, 0, output_tensor)

    #     # Assert that axonsDict contains expected keys and bias values
    #     self.assertIn('a0', axonsDict)  # Check if axon ID 'a0' exists in axonsDict
    #     self.assertIn('a1', axonsDict)  # Check if axon ID 'a1' exists in axonsDict

    # def test_linearBiasAxons(self):
    #     # Test linearBiasAxons method with a simple linear layer
    #     linear_layer = nn.Linear(in_features=4, out_features=2)
    #     input_tensor = torch.randn(1, 4)  # Batch size: 1, Input features: 4
    #     output_tensor = linear_layer(input_tensor)
    #     axonsDict = {}
    #     self.converter.linearBiasAxons(linear_layer, axonsDict, 0, output_tensor)

    #     # Assert that axonsDict contains expected keys and bias values
    #     self.assertIn('a0', axonsDict)  # Check if axon ID 'a0' exists in axonsDict
    #     self.assertIn('a1', axonsDict)  # Check if axon ID 'a1' exists in axonsDict
        
# Create a test loader
loader = unittest.TestLoader()

# Load tests from the test class
suite = loader.loadTestsFromTestCase(TestLayerToCRIConverter)

# Use TextTestRunner to run the tests
unittest.TextTestRunner().run(suite)

# if __name__ == '__main__':
#     unittest.main()

E
ERROR: test_conv2dToCRI (__main__.TestLayerToCRIConverter)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/vr/5qw6zp9500gd5_dc2ts4bk8r0000gn/T/ipykernel_31907/3221027896.py", line 16, in test_conv2dToCRI
    self.converter.conv2dToCRI(input_tensor, output_tensor, conv_layer, 0, neuronsDict=neuronsDict)
  File "/var/folders/vr/5qw6zp9500gd5_dc2ts4bk8r0000gn/T/ipykernel_31907/3156014874.py", line 60, in conv2dToCRI
    Hi, Wi = inputs.shape
ValueError: too many values to unpack (expected 2)

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (errors=1)


<unittest.runner.TextTestResult run=1 errors=1 failures=0>